### With Excel and pdf parser and handling for multiple directories


# All imports and inits

In [8]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from tkinter import scrolledtext, messagebox
from transformers import AutoModel, AutoTokenizer
# from pinecone import Pinecone, ServerlessSpec
import pinecone
from pinecone import (
    Pinecone,
    ServerlessSpec,
    CloudProvider,
    AwsRegion,
    VectorType
)

import os
import requests
import PyPDF2
import textwrap
import numpy as np
import streamlit as st
import tkinter as tk
import gradio as gr
from typing import List, Tuple
import concurrent.futures
import glob
import pandas as pd
import numpy as np
# Important: Import pinecone-client properly
# Load environment variables from .env file
load_dotenv()

DATA_PATH = os.getenv("DATA_PATH")
PINECONE_API = os.getenv("PINECONE_API")
PINECONE_ENV = os.getenv("PINECONE_ENV")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print("PINECONE_API", PINECONE_API)


# Groq API settings
GROQ_EMBED_URL = "https://api.groq.com/openai/v1/embeddings"
GROQ_CHAT_URL = "https://api.groq.com/openai/v1/chat/completions"
EMBEDDING_MODEL = "llama3-405b-8192-embed"
LLM_MODEL = "llama3-70b-8192"


# Configure headers for Groq API requests
GROQ_HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}


PINECONE_API pcsk_4bLR9o_3crxHE9zjHW76VdRnBPi2Xo794pQnKSifnRfQ9iQc6U3iqeqeyVEZ3RjBPYtoD4


# PDF Parser


In [3]:
def pdf_load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()


# documents = pdf_load_documents()
# documents


# def extract_text_from_pdf(pdf_path: str) -> str:
#     """Extract text from a PDF file."""
#     with open(pdf_path, 'r') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         text = ""
#         for page_num in range(len(pdf_reader.pages)):
#             page = pdf_reader.pages[page_num]
#             text += page.extract_text() + "\n"
#     return text
# extract_text_from_pdf(DATA_PATH)


## PDF and Excel Parser (Maqbool)

In [4]:
# def load_documents():
#     documents = []

#     # Load PDFs
#     pdf_loader = PyPDFDirectoryLoader(DATA_PATH)
#     pdf_documents = pdf_loader.load()
#     documents.extend(pdf_documents)

#     # Load Excel files
#     excel_files = glob.glob(os.path.join(DATA_PATH, "*.xlsx"))
#     if excel_files:
#         excel_documents = [pd.read_excel(file) for file in excel_files]
#         documents.extend(excel_documents)

#     if not documents:
#         raise FileNotFoundError(f"No PDF or Excel files found in {DATA_PATH}")

#     return documents

# # Load documents
# documents = load_documents()

# print(documents)
# # Display the first document (PDF or Excel)
# if isinstance(documents[0], pd.DataFrame):
#     print(documents[0].head())  # Print first few rows if it's an Excel file
# else:
#     print(documents[0])  # Print the content if it's a PDF document


def load_documents():
    documents = []
    file_names = []

    # Load PDFs
    pdf_loader = PyPDFDirectoryLoader(DATA_PATH)
    pdf_documents = pdf_loader.load()
    documents.extend(pdf_documents)
    
    # Get PDF filenames
    pdf_files = glob.glob(os.path.join(DATA_PATH, "*.pdf"))
    print(f"Found PDF files: {pdf_files}")  # Debug print
    file_names.extend([os.path.basename(file) for file in pdf_files])

    # Load Excel files
    excel_files = glob.glob(os.path.join(DATA_PATH, "*.xlsx"))
    print(f"Found Excel files: {excel_files}")  # Debug print
    if excel_files:
        for file in excel_files:
            documents.append(pd.read_excel(file))
            file_names.append(os.path.basename(file))

    if not documents:
        raise FileNotFoundError(f"No PDF or Excel files found in {DATA_PATH}")

    return documents, file_names

# Load documents
documents, file_names = load_documents()

# Print file names
print("Files parsed:")
for name in file_names:
    print(f"- {name}")

print(f"\nTotal documents loaded: {len(documents)}")



Found PDF files: []
Found Excel files: []
Files parsed:

Total documents loaded: 360


### Single Directory Parser

In [ ]:
def load_documents():
    """
    Load PDF and Excel files from DATA_PATH.
    Returns a list of documents with content and metadata and a list of filenames.
    """
    documents = []
    file_names = []

    # Load PDFs
    pdf_loader = PyPDFDirectoryLoader(DATA_PATH)
    pdf_docs = pdf_loader.load()
    for doc in pdf_docs:
        documents.append({
            "content": doc.page_content,
            "metadata": {"source": doc.metadata.get("source", "unknown"), "file_type": "pdf"}
        })
        file_names.append(doc.metadata.get("source", "unknown"))

    # Load Excel files
    excel_files = glob.glob(os.path.join(DATA_PATH, "*.xlsx"))
    for file in excel_files:
        df = pd.read_excel(file)
        headers = df.columns.tolist()
        for _, row in df.iterrows():
            content = " ".join([f"{col}: {str(row[col])}" for col in headers])
            documents.append({
                "content": content,
                "metadata": {"source": file, "file_type": "excel"}
            })
        file_names.append(file)

    if not documents:
        print(f"No PDF or Excel files found in {DATA_PATH}")
    else:
        print(f"Loaded {len(documents)} documents")

    return documents, file_names

# Load documents
documents, file_names = load_documents()

# Print file names
print("Files parsed:")
for name in file_names:
    print(f"- {name}")

print(f"\nTotal documents loaded: {len(documents)}")

### Multiple Directory Parser

In [26]:
# def load_documents():
#     """
#     Load PDF and Excel files from DATA_PATH and its subdirectories.
#     Returns a list of documents with content and metadata and a list of filenames.
#     """
#     documents = []
#     file_names_set = set()

#     # Load PDFs from all subdirectories
#     pdf_loader = PyPDFDirectoryLoader(DATA_PATH, recursive=True)
#     pdf_docs = pdf_loader.load()
#     for doc in pdf_docs:
#         source = doc.metadata.get("source", "unknown")
#         if source not in file_names_set:
#             documents.append({
#                 "content": doc.page_content,
#                 "metadata": {"source": source, "file_type": "pdf"}
#             })
#             file_names_set.add(source)

#     # Load Excel files from all subdirectories
#     for root, _, files in os.walk(DATA_PATH):
#         for file in files:
#             if file.endswith(".xlsx"):
#                 file_path = os.path.join(root, file)
#                 if file_path not in file_names_set:
#                     df = pd.read_excel(file_path)
#                     headers = df.columns.tolist()
#                     for _, row in df.iterrows():
#                         content = " ".join([f"{col}: {str(row[col])}" for col in headers])
#                         documents.append({
#                             "content": content,
#                             "metadata": {"source": file_path, "file_type": "excel"}
#                         })
#                     file_names_set.add(file_path)

#     if not documents:
#         print(f"No PDF or Excel files found in {DATA_PATH}")
#     else:
#         print(f"Loaded {len(documents)} documents")

#     return documents, list(file_names_set)

# # Load documents
# documents, file_names = load_documents()

# # Print file names
# print("Files parsed:")
# for name in file_names:
#     print(f"- {name}")

# print(f"\nTotal documents loaded: {len(documents)}")
# #documents


def load_documents():
    """
    Load PDF and Excel files from DATA_PATH and its subdirectories.
    Returns a list of documents with content and metadata, and a list of filenames.
    """
    documents = []
    file_names = []
    processed_files = []

    # Load PDFs from all subdirectories
    pdf_loader = PyPDFDirectoryLoader(DATA_PATH, recursive=True)
    pdf_docs = pdf_loader.load()
    print("PDF DOCS len: \n\n",len(pdf_docs))
    #print("PDF DOCS: \n\n",pdf_docs)
    count = 0
    for doc in pdf_docs:
        source = doc.metadata.get("source", "unknown")
        if source not in processed_files:
            documents.append({
                "content": doc.page_content,
                "metadata": {"source": source, "file_type": "pdf"}
            })
            #print(f"PARSED DATA {source}: \n",documents[-1])
    
            processed_files.append(source)
            file_names.append(source)
            #print(f"Parsed PDF file : {source}")
        # print("COUNT: ", count)
        count+=1
   # return 

    # Load Excel files from all subdirectories
    for root, _, files in os.walk(DATA_PATH):
        for file in files:
            if file.endswith(".xlsx"):
                file_path = os.path.join(root, file)
                if file_path not in processed_files:
                    df = pd.read_excel(file_path)
                    headers = df.columns.tolist()
                    for _, row in df.iterrows():
                        content = " ".join([f"{col}: {str(row[col])}" for col in headers])
                        documents.append({
                            "content": content,
                            "metadata": {"source": file_path, "file_type": "excel"}
                        })
                        #print("PARSED DATA: \n\n\n",documents[-1])
                    processed_files.append(file_path)
                    file_names.append(file_path)
                    print(f"Parsed Excel file: {file_path}")

    if not documents:
        print(f"No PDF or Excel files found in {DATA_PATH}")
    else:
        print(f"Loaded {len(documents)} documents")
    # Print file names
    print("Files parsed:")
    for name in processed_files:
        print(f"- {name}")
    return documents, file_names

# Load documents
documents, file_names = load_documents()



print(f"\nTotal documents loaded: {len(documents)}")

PDF DOCS len: 

 369
Parsed Excel file: D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\Course 1 - Introduction to Surgical Technology\Assessments\Exam 01.xlsx
Parsed Excel file: D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\Course 1 - Introduction to Surgical Technology\Assessments\Lesson 01 - Quiz.xlsx
Parsed Excel file: D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\Course 1 - Introduction to Surgical Technology\Assessments\Lesson 02 - Quiz.xlsx
Parsed Excel file: D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\Course 1 - Introduction to Surgical Technology\Assessments\Lesson 03 - Quiz.xlsx
Parsed Excel file: D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\Course 1 - Introduction to Surgical Technology\Assessments\Lesson 04 - Quiz.xlsx
Parsed Excel file: D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\Course 1 - Introduction to Surgical Technology\Assessments\Lesson 05 - Quiz.xlsx
Parsed Excel file: D:\Disrupt Labs

# Text Splitting \ Chunking using Langchain

In [36]:


# def split_documents(documents):
#     text_splitter = RecursiveCharacterTextSplitter(
#         # Set a really small chunk size, just to show.
#         chunk_size=800,
#         chunk_overlap=80,
#         length_function=len,
#         is_separator_regex=False  # considers separators like '\n\n'if true
#     )
#     docs = text_splitter.split_documents(documents)
#     return docs


# chunks = split_documents(documents)
# chunks 

def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False  # considers separators like '\n\n' if true
    )
    # Assuming each document is a dictionary with 'content' and 'metadata'
    docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc['content'])
        for i, chunk in enumerate(chunks):
            docs.append({
                "content": chunk,
                "metadata": {
                    **doc['metadata'],
                    "chunk_id": i
                }
            })
    return docs

# Split documents into chunks
chunks = split_documents(documents)
print(len(chunks))

147


# Init Pinecone

In [31]:
pc = Pinecone(api_key=PINECONE_API)
print(PINECONE_API)


#  --------------- initialize pinecone -----------------------------
# pc.create_index_for_model(
#     name="test-index",
#     cloud="aws",
#     region="us-east-1",
#     embed={
#         "model":"llama-text-embed-v2",
#         "field_map":{"text": "page_content"}
#     }
# )


pcsk_4bLR9o_3crxHE9zjHW76VdRnBPi2Xo794pQnKSifnRfQ9iQc6U3iqeqeyVEZ3RjBPYtoD4


### When to Use What:
**Use Upsert:**

When you're adding new vectors or want to replace existing vectors with new data (including changing the vector values).
When you need to add a completely new document or vector.
When you want to update both the vector values and metadata.

**Use Update:**

When you're only modifying the metadata of an existing vector.
When the vector values (embeddings) themselves are correct and only extra information like text, author, or document-related metadata needs to be updated.
Summary:
Upsert: Adds or replaces both the vector values and metadata. Use when inserting or completely replacing data.
Update: Modifies the metadata without changing the vector values. Use when the vectors are correct, but metadata needs an update.
For your case, if you just want to add or update the page_content or any other metadata for existing vectors, use update. If you want to re-upload vectors with new embeddings or metadata, use upsert.









## Creating Embeddings Via AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en'  and Upsert each to Pinecone one by one


In [35]:
# Connect to the index
index = pc.Index("ai-coach")
#index = pc.Index("ahsan-400pg-pdf-doc-test")


embedding_model = AutoModel.from_pretrained(
    'jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
# user_query = "user query"
# Function to generate embeddings without tokenization


def get_embedding(data):
    embeddings = embedding_model.encode(data).tolist()
    return embeddings

# def upsert_chunks_to_pinecone(index, chunks):
#   count = 1
#   for chunk in chunks:
#     #embedding = embedding_model.encode(chunk.page_content).tolist()
#     embedding = get_embedding(chunk.page_content)
#     # Extract metadata
#     metadata = chunk.metadata
#     text = chunk.page_content
#     # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
#     vector_id = f"vec_{count}"

#     # Upsert the embedding along with its metadata
#     index.upsert(vectors=[(vector_id, embedding, metadata, text)])

#     print(f"Embedding {count} upserted to Pinecone with metadata")
#     count += 1
#       # Ensure data is written immediately
#   print(f"All {count} Embeddings have been upserted to pinecone")


# def upsert_chunks_to_pinecone(index, chunks):
#     count = 1
#     for chunk in chunks:
#         # Get the embedding for the chunk
#         embedding = get_embedding(chunk.page_content)

#         # Extract metadata and add text as part of the metadata
#         metadata = chunk.metadata
#         metadata["text"] = chunk.page_content  # Store text in metadata

#         # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
#         vector_id = f"vec_{count}"

#         # Upsert the embedding along with its metadata
#         index.upsert(vectors=[(vector_id, embedding, metadata)])

#         print(f"Embedding {count} upserted to Pinecone with metadata")
#         count += 1

#     print(f"All {count-1} Embeddings have been upserted to Pinecone")


# upsert_chunks_to_pinecone(index, chunks)

def upsert_chunks_to_pinecone(index, chunks):
    count = 1
    for chunk in chunks:
        # Ensure the chunk has the correct structure
        content = chunk.get('content')
        metadata = chunk.get('metadata', {})

        # Get the embedding for the chunk
        embedding = get_embedding(content)

        # Add the text as part of the metadata
        metadata['text'] = content  # Store text in metadata

        # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
        vector_id = f"vec_{count}"

        # Upsert the embedding along with its metadata
        index.upsert(vectors=[(vector_id, embedding, metadata)])

        print(f"Embedding {count} upserted to Pinecone with metadata")
        count += 1

    print(f"All {count-1} embeddings have been upserted to Pinecone")

# Example usage
# Assuming `index` is your Pinecone index and `chunks` is the list of chunked documents
upsert_chunks_to_pinecone(index, chunks)

# query_embeddings = embedding_model.encode(user_query).tolist()
# query_embeddings


Embedding 1 upserted to Pinecone with metadata
Embedding 2 upserted to Pinecone with metadata
Embedding 3 upserted to Pinecone with metadata
Embedding 4 upserted to Pinecone with metadata
Embedding 5 upserted to Pinecone with metadata
Embedding 6 upserted to Pinecone with metadata
Embedding 7 upserted to Pinecone with metadata
Embedding 8 upserted to Pinecone with metadata
Embedding 9 upserted to Pinecone with metadata
Embedding 10 upserted to Pinecone with metadata
Embedding 11 upserted to Pinecone with metadata
Embedding 12 upserted to Pinecone with metadata
Embedding 13 upserted to Pinecone with metadata
Embedding 14 upserted to Pinecone with metadata
Embedding 15 upserted to Pinecone with metadata
Embedding 16 upserted to Pinecone with metadata
Embedding 17 upserted to Pinecone with metadata
Embedding 18 upserted to Pinecone with metadata
Embedding 19 upserted to Pinecone with metadata
Embedding 20 upserted to Pinecone with metadata
Embedding 21 upserted to Pinecone with metadata
E

# Update Vectors Function

In [7]:
def update_pinecone_chunks(index, chunks):
    count = 1
    for chunk in chunks:
        # Get updated embedding
        embedding = get_embedding(chunk.page_content)

        # Extract metadata and page content
        metadata = chunk.metadata
        text = chunk.page_content

        # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
        vector_id = f"vec_{count}"

        # Update the embedding and metadata
        index.update(id=vector_id, values=embedding, set_metadata=metadata)

        print(f"Embedding {count} updated in Pinecone with new metadata")
        count += 1

    print(f"All {count-1} embeddings have been updated in Pinecone")

# update_pinecone_chunks(index, chunks)


Since your application is designed to answer a wide range of student queries and suggest relevant material, you want to retrieve enough content to cover different facets of a topic without overwhelming the LLM with too much information.

# Starting Point:
- A common starting point is to set top_k between **5 and 10.**
- **top_k=5:** This can work well if your curated content is highly relevant and precise, ensuring that the top 5 matches are very close to the query.
-  **top_k=10:** If you want the coach to consider a broader range of content—perhaps to provide diverse perspectives or cover a topic more comprehensively—increasing top_k to around 10 might be beneficial.

# Experiment and Adjust:
- The “best” value depends on factors such as the diversity of your content, how densely your data covers the topics, and the quality of the embedding matches. It’s a good idea to experiment with different top_k values and evaluate the quality and relevance of the responses in your specific


# Query Pinecone


In [40]:
# Function to query Pinecone index using embeddings
def query_pinecone(embedding):
    # Use keyword arguments to pass the embedding and other parameters
    result = index.query(vector=embedding, top_k=20, include_metadata=True)
    return result['matches']


# Query Groq Inference

In [41]:
# Function to query Groq LLM
def query_groq(prompt: str) -> str:
    response = requests.post(
        GROQ_CHAT_URL,
        headers=GROQ_HEADERS,
        json={
            "model": LLM_MODEL,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.5,
            "max_tokens": 8192  # max from groq website
        }
    )

    if response.status_code != 200:
        raise Exception(f"Error querying Groq: {response.text}")

    return response.json()["choices"][0]["message"]["content"]


# Tokenizer to count number of tokens
tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v2-base-en")


def count_tokens(text: str) -> int:
    # Encode the text into tokens
    tokens = tokenizer.encode(text)
    return len(tokens)


# Process User Query

# Gradio GUI TEST

In [42]:
# system_message = f"""
#     You are a knowledgeable and friendly coach. Your goal is to help students understand concepts in a detailed and easy-to-understand manner. 
#     Be patient, ask guiding questions, and provide step-by-step explanations where needed. Adapt your responses to the student's knowledge level 
#     and help them build confidence in their learning. Refer relevant material to the student and encourage them to explore further.

#     Based on the context and the student's question, provide a thoughtful and detailed explanation. Encourage them to think about the topic and 
#     offer further guidance if needed.
#     """

# def gradio_interface(prompt,history =[]):
#     output = process_user_query(prompt,history)
#     history.append((prompt,output))
#     return history

# gr.Interface(fn=gradio_interface, inputs= ['text',"state"], outputs=["chatbot","state"]).launch(debug=True,share=True)
    

# ------------------------------------------- WORKING 1 -------------------------------------------

# # Function to be used by Gradio for handling the query
# def gradio_process(user_query):
#     response = process_user_query(user_query, conversation_history)
#     return response

# # Create Gradio interface
# interface = gr.Interface(fn=gradio_process, inputs="text", outputs="text", title="RAG-based Coaching System")

# # Launch Gradio app
# interface.launch()
# ------------------------------------------- WORKING 2 -------------------------------------------

# Initialize empty conversation history (list of tuples)
# conversation_history = []

# def process_user_query(user_query: str, conversation_history: list):
#     print(f"User Query Tokens: {count_tokens(user_query)}")

#     # Generate embedding and get relevant context
#     embedding = get_embedding(user_query)
#     relevant_chunks = query_pinecone(embedding)
#     context = "\n".join(chunk['metadata']["text"] for chunk in relevant_chunks)
#     print("CONTEXT:", context)

#     # Format conversation history for the prompt
#     history_str = "\n".join(
#         f"User: {user}\nCoach: {response}" 
#         for user, response in conversation_history
#     )

#     # Create structured prompt
#     prompt = f"""You are a knowledgeable and friendly coach. Follow these guidelines:
#     1. Provide clear, step-by-step explanations
#     2. Ask guiding questions to encourage critical thinking
#     3. Adapt to the student's knowledge level
#     4. Use examples from the provided context when relevant

#     Context from learning materials:
#     {context}

#     Conversation history:
#     {history_str}

#     New student question:
#     "{user_query}"

#     Provide a helpful response:"""

#     # Get LLM response
#     groq_response = query_groq(prompt)
#     print(f"Response Tokens: {count_tokens(groq_response)}")

#     # Return updated history with new interaction
#     return conversation_history + [(user_query, groq_response)]

# # Gradio Interface
# with gr.Blocks() as interface:
#     gr.Markdown("# 🧑‍🏫 AI Coaching Assistant")
#     gr.Markdown("Welcome! I'm here to help you learn. Type your question below.")
    
#     # State management
#     chat_history = gr.State(conversation_history)
    
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500)
#         with gr.Column(scale=0.5):
#             context_display = gr.Textbox(label="Relevant Context", interactive=False)

#     user_input = gr.Textbox(label="Your Question", placeholder="Type here...")
    
#     with gr.Row():
#         submit_btn = gr.Button("Submit", variant="primary")
#         undo_btn = gr.Button("Undo Last")
#         clear_btn = gr.Button("Clear History")

#     def handle_submit(user_input, history):
#         if not user_input.strip():
#             return gr.update(), history, ""
        
#         # Process query and update history
#         new_history = process_user_query(user_input, history)
        
#         # Get latest context for display
#         latest_context = "\n".join([chunk['metadata']["text"] for chunk in query_pinecone(
#             get_embedding(user_input)
#         )][:3])  # Show top 3 context snippets
        
#         return "", new_history, latest_context

#     # Component interactions
#     submit_btn.click(
#         handle_submit,
#         [user_input, chat_history],
#         [user_input, chat_history, context_display]
#     ).then(
#         lambda x: x,
#         [chat_history],
#         [chatbot]
#     )

#     undo_btn.click(
#         lambda history: history[:-1] if history else [],
#         [chat_history],
#         [chat_history]
#     ).then(
#         lambda x: x,
#         [chat_history],
#         [chatbot]
#     )

#     clear_btn.click(
#         lambda: [],
#         None,
#         [chat_history]
#     ).then(
#         lambda: ([], ""),
#         None,
#         [chatbot, context_display]
#     )

# interface.launch(share=True)
# Just change the launch command to:
#interface.launch(share=True, auth=("username", "password"))  # Add basic auth


# self hosting

# # Run with:
# interface.launch(
#     server_name="0.0.0.0",
#     server_port=7860,
#     show_error=True
# )


# ------------------------------------------- WORKING 3 Enter key submits user query -------------------------------------------
# Initialize empty conversation history (list of tuples)
conversation_history = []

def process_user_query(user_query: str, conversation_history: list):
    print(f"User Query Tokens: {count_tokens(user_query)}")

    # Generate embedding and get relevant context
    embedding = get_embedding(user_query)
    relevant_chunks = query_pinecone(embedding)
    context = "\n".join(chunk['metadata']["text"] for chunk in relevant_chunks)
    print("CONTEXT:", context)

    # Format conversation history for the prompt
    history_str = "\n".join(
        f"User: {user}\nCoach: {response}" 
        for user, response in conversation_history
    )

    # Create structured prompt
    prompt = f"""You are a knowledgeable and friendly coach. Follow these guidelines:
    1. Provide clear, step-by-step explanations
    2. Ask guiding questions to encourage critical thinking
    3. Adapt to the student's knowledge level
    4. Use examples from the provided context when relevant

    Context from learning materials:
    {context}

    Conversation history:
    {history_str}

    New student question:
    "{user_query}"

    Provide a helpful response:"""

    # Get LLM response
    groq_response = query_groq(prompt)
    print(f"Response Tokens: {count_tokens(groq_response)}")

    # Return updated history with new interaction
    return conversation_history + [(user_query, groq_response)]

# Gradio Interface
with gr.Blocks() as interface:
    gr.Markdown("# 🧑‍🏫 AI Coaching Assistant")
    gr.Markdown("Welcome! I'm here to help you learn. Type your question below.")
    
    # State management
    chat_history = gr.State(conversation_history)
    
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        with gr.Column(scale=0.5):
            context_display = gr.Textbox(label="Relevant Context", interactive=False)

    user_input = gr.Textbox(label="Your Question", placeholder="Type here...")
    
    with gr.Row():
        submit_btn = gr.Button("Submit", variant="primary")
        undo_btn = gr.Button("Undo Last")
        clear_btn = gr.Button("Clear History")

    def handle_submit(user_input, history):
        if not user_input.strip():
            return gr.update(), history, ""
        
        # Process query and update history
        new_history = process_user_query(user_input, history)
        
        # Get latest context for display
        latest_context = "\n".join([chunk['metadata']["text"] for chunk in query_pinecone(
            get_embedding(user_input)
        )][:3])  # Show top 3 context snippets
        
        return "", new_history, latest_context

    # Component interactions
    submit_btn.click(
        handle_submit,
        [user_input, chat_history],
        [user_input, chat_history, context_display]
    ).then(
        lambda x: x,
        [chat_history],
        [chatbot]
    )
    
    # Add submit on Enter key press
    user_input.submit(
        handle_submit,
        [user_input, chat_history],
        [user_input, chat_history, context_display]
    ).then(
        lambda x: x,
        [chat_history],
        [chatbot]
    )

    undo_btn.click(
        lambda history: history[:-1] if history else [],
        [chat_history],
        [chat_history]
    ).then(
        lambda x: x,
        [chat_history],
        [chatbot]
    )

    clear_btn.click(
        lambda: [],
        None,
        [chat_history]
    ).then(
        lambda: ([], ""),
        None,
        [chatbot, context_display]
    )

interface.launch(share=True)


C:\Users\ABC\AppData\Local\Temp\ipykernel_3920\262375163.py:208: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)
d:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\env\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://ea159ff23b0ef964fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


User Query Tokens: 12
CONTEXT: History	of	Surgical	Technology
(0:08	-	0:20)
Hello	everyone,	and	welcome	back	to	our	channel.	A	while	ago,	we	made	a	video	on	the	history
of	surgery.	If	you	haven't	watched	that,	go	do	it	after	this,	which	you	all	loved,	so	we	thought
we'd	make	more	surgery-related	content	for	you	guys.
(0:21	-	0:32)
That	is	why	today	we	have	a	video	on	the	history	of	surgical	technology.	It's	all	stupid
interesting,	and	we'll	be	talking	about	a	role	in	hospitals	that	is	so	important,	but	also	so
underrated.	So,	let's	begin.
(0:33	-	0:48)
To	the	operating	theatre,	don't	forget	your	gloves.	What	is	surgical	technology?	To	start	off
with,	let's	understand	what	surgical	technology	is.	You	might	hear	the	word	technology	and
think	about	cool	gadgets	and	smart	systems,	but	that's	not	exactly	right.
(0:48	-	1:07)
Medicolegal Aspects of 
Surgical Technology
C1L5	-	Ashley	Hay
(0:06	-	0:18)
Welcome	back.	We	are	on	the	last	lesson	for	course	one.	So	course	one,	lesson	five,	we're
go